<img align="center" src="figures/course.png" width="800">

#                                    16720 (B) Bag of Visual Words - Assignment 2

     Instructor: Kris Kitani                   TAs:Paritosh (Lead), Rawal, Yan, Zen, Wen-Hsuan, Qichen

In [1]:
import nbimporter
import numpy as np
import skimage
import multiprocessing
import threading
import queue
import os,time
import math
from ipynb.fs.defs.p1 import get_visual_words
from ipynb.fs.defs.p2 import get_image_feature, distance_to_set

## For Autograding P3, ensure uploading `conf_matrix.npy`.

### Quantitative Evaluation

#### Calculating confusion matrix
Qualitative evaluation is all well and good (and very important for diagnosing performance gains and losses), but we want some hard numbers.

Load the corresponding test images and their labels, and compute the predicted labels of each, i.e., compute its distance to every image in training set and return the label with least distance difference as the predicted label. To quantify the accuracy, you will compute a confusion matrix $C$: given a classification problem, the entry $C(i,j)$ of a confusion matrix counts the number of instances of class $i$ that were predicted as class $j$. When things are going well, the elements on the diagonal of $C$ are large, and the off-diagonal elements are small. Since there are 8 classes, $C$ will be $8 \times 8$. The accuracy, or percent of correctly classified images, is given by the trace of $C$ divided by the sum of $C$.

#### Q3.1.1 (10 Points -> 5 Autograder + 5 WriteUp)
Implement the function
```
            def evaluate_recognition_system():
```
that tests the system and outputs the confusion matrix.

Report the confusion matrix and accuracy for your results in your write-up. This does not have to be formatted prettily: if you are using LaTeX, you can simply copy/paste it into a $verbatim$ environment. Additionally, do not worry if your accuracy is low: with 8 classes, chance is $12.5\%$. To give you a more sensible number, a reference implementation _with_ spatial pyramid matching gives an overall accuracy of around $50\%$.

In [5]:
from ipynb.fs.defs.p2 import get_feature_from_wordmap_SPM

def evaluate_recognition_one_image(file_path, dictionary, layer_num, K, trained_features, trained_labels):
    # helper_function
    # YOUR CODE HERE
    
    #read image data
    image = skimage.io.imread(file_path)
    image = image.astype('float') / 255
    
    # get word map
    wordmap = get_visual_words(image, dictionary)
    image_hist = get_feature_from_wordmap_SPM(wordmap, layer_num, K)
    similarity_scores = distance_to_set(image_hist, trained_features)
    
    # find the index of the highest score
    nearest_image_idx = np.where(similarity_scores == np.max(similarity_scores))[0][0]
    pred_label = trained_labels[nearest_image_idx]
    
    # raise NotImplementedError()
    return [file_path, pred_label, nearest_image_idx]


def evaluate_recognition_system(num_workers=2):
    '''
    Evaluates the recognition system for all test images and returns the confusion matrix.

    [input]
    * num_workers: number of workers to process in parallel

    [output]
    * conf: numpy.ndarray of shape (8,8)
    * accuracy: accuracy of the evaluated system
    '''
    '''
    HINTS
    (1) You may wish to use multiprocessing to improve speed (NO Extra Points)
    (2) You may create helper function (in the same cell) to enable multiprocessing
    (3) Think Nearest Neighbor -> assign label using element closest in train set
    '''
    
    test_data = np.load("./data/test_data.npz")
    trained_system = np.load("trained_system.npz")
    
    image_names = test_data['files']
    test_labels = test_data['labels']
    num_images = image_names.shape[0]

    trained_features = trained_system['features']
    trained_labels = trained_system['labels']
    dictionary = trained_system['dictionary']
    SPM_layer_num = trained_system['SPM_layer_num']
    SPM_layer_num = int(SPM_layer_num)
    K = dictionary.shape[0]

    print("Trained features shape: ", trained_features.shape)
    
    # ----- TODO -----
    '''
    HINTS:
    1.> Think almost exactly similar to Q1.2.2
    2.> Create a list of arguments and use multiprocessing library
    3.> We can define a helper function which can take in the arguments (file_path, dictionary, SPM_layer_num,
        trained_features,...) as input and return (file_path, label, nearest neighbor index)
    4.> We can use python dictionary and file_path to have the output in correct order
    '''
    # YOUR CODE HERE
    
    results = []
    for i in range(num_images):
        file_path = './data/' + image_names[i]
        file_label = test_labels[i]
        results.append(evaluate_recognition_one_image(file_path, dictionary, SPM_layer_num, K, trained_features, trained_labels))
        if (i%10 == 0): print(int(i/num_images*100), '%')
    
    # raise NotImplementedError()
    '''
    HINTS:
    1.> Can use the file_name (path) to place the labels back in original order of input to multiprocessing
    '''
    # ordered_labels = [None] * len(pred_labels)
    # YOUR CODE HERE
    
    # multiprocessing not used, so the labels are in original order
    ordered_labels = np.array([result[1] for result in results])
    
    # raise NotImplementedError()
    ordered_labels = np.array(ordered_labels, dtype=int)
    
    print("Predicted labels shape: ", ordered_labels.shape)
    
    '''
    HINT:
    1.> Compute the confusion matrix (8x8)
    2.> Remember to save and upload the confusion matrix
    '''
    # YOUR CODE HERE
    
    conf_matrix = np.zeros((8, 8))
    for idx in range(len(results)):
        i = test_labels[idx]
        j = ordered_labels[idx]
        conf_matrix[i, j] += 1
    accuracy = np.diag(conf_matrix).sum() / conf_matrix.sum()
    
    print("accuracy = ", accuracy)
    
    # raise NotImplementedError()
    np.save("./conf_matrix.npy",conf_matrix)
    return conf_matrix, accuracy

# evaluate_recognition_system()

Trained features shape:  (1000, 4200)
0 %
6 %
12 %
18 %
25 %
31 %
37 %
43 %
50 %
56 %
62 %
68 %
75 %
81 %
87 %
93 %
Predicted labels shape:  (160,)
accuracy =  0.625


(array([[13.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
        [ 0., 14.,  0.,  0.,  0.,  1.,  3.,  0.],
        [ 0.,  1., 13.,  5.,  2.,  0.,  1.,  3.],
        [ 2.,  2.,  2., 15.,  0.,  1.,  0.,  4.],
        [ 2.,  1.,  0.,  0., 10.,  0.,  0.,  0.],
        [ 2.,  1.,  0.,  1.,  6., 11.,  3.,  0.],
        [ 0.,  4.,  0.,  0.,  3.,  0., 12.,  2.],
        [ 0.,  2.,  2.,  2.,  1.,  0.,  0., 12.]]),
 0.625)

<font color="blue">**Submit the Confusion Matrix and the Accuracy Value in the WriteUp**</font>

#### Q3.1.2 (5 points WriteUp):
<font color="blue"> As there are some classes/samples that are more difficult to classify than the rest using the bags-of-words approach, they are more easily classified incorrectly into other categories. **List some of these classes/samples and discuss why they are more difficult in your write-up.** </font>

### Q3.1.3 [Extra Credit](10 points) Manually Graded:

Now that you have seen how well your recognition system can perform on a set of real images, you can experiment with different ways of improving this baseline system. Here are a few suggestions:

* Hyperparameter Tuning: here is a list of hypterparameters in the system that you can tune to get better performance for your system:
        
        * `filter_scales`: a list of filter scales used in extracting filter response;
        * `K`: the number of visual words and also the size of the dictionary;
        * `alpha`: the number of sampled pixels in each image when creating the dictionary;
        * `L`: the number of spatial pyramid layers used in feature extraction.
        
* Image manipulation: Try using image augmentation techniques such as random-crop, flipping, etc. to obtain more training data for your system. You can also try resizing the images, subtracting the mean color, etc. 

* Better classifier: in part 2 we used the nearest neighbor classifier to classify test images. However, with our extracted SPM features from training images, we can use other classifiers such as multi-class logistic regression, multi-class support vector machine, etc. to gain better performance. For this, you can use implementation of these algorithms from `scipy`.


Tune the system you build to reach around 65\% accuracy on the provided test set (``data/test_data.npz``). <font color="blue">**In your writeup, document what you did to achieve such performance: (1) what you did, (2) what you expected would happen, and (3) what actually happened.** Also, include a file called ``custom.py/ipynb`` for running your code. </font>

### Q3.1.4 [Extra Credit] (10 points):
**Inverse Document Frequency:** With the bag-of-word model, image recognition is similar to classifying a document with words. In document classification, inverse document frequency (IDF) factor is incorporated which diminishes the weight of terms that occur very frequently in the document set. For example, because the term "the" is so common, this will tend to incorrectly emphasize documents which happen to use the word "the" more frequently, without giving enough weight to the more meaningful terms.

In the homework, the histogram we computed only considers the term frequency (TF), i.e.  the number of times that word occurs in the word map.  Now we want to weight the word by its inverse document frequency.  The IDF of a word is defined as:

\begin{align*} IDF_w &= log \frac{T}{|\{d: w \in
d\}|}\\ \end{align*}

Here, $T$ is number of all training images, and $|\{d:w\in d\}|$ is the number of images $d$ such that $w$ occurs in that image.

Write a function ``compute_IDF`` to compute a vector ``IDF`` of size $1\times K$ containing IDF for all visual words, where K is the dictionary size. Save the extracted ``IDF`` in ``idf.npy``. Then write another function ``evaluate_recognition_System_IDF`` that makes use of the ``IDF`` vector in the recognition process. You can use either nearest neighbor or anything you have from q3.1.4 as your classifier.

<font color="blue">**In your writeup: How does Inverse Document Frequency affect the performance? Better or worse? Explain your reasoning?**

**Remember to submit idf.npy along with the submission**</font>

In [ ]:
def compute_IDF():
    # YOUR CODE HERE
    raise NotImplementedError()
    pass

def evaluate_recognition_System_IDF():
    # YOUR CODE HERE
    raise NotImplementedError()
    pass